# 파이썬 2+3 호환 코딩

파이썬 2와 3 환경에서 모두 실행 가능한 코드를 짜기.

## 들어가기

파이썬 2와 3은 알게 모르게 큰 차이가 있다.

In [1]:
# -*- coding: utf-8 -*-

import sys

print '파이썬 버젼: ', sys.version     # 파이썬 2
print '\n'
print('파이썬 버젼: ', sys.version)    # 파이썬 3?

파이썬 버젼:  2.7.15+ (default, Nov 27 2018, 23:36:35) 
[GCC 7.3.0]


('\xed\x8c\x8c\xec\x9d\xb4\xec\x8d\xac \xeb\xb2\x84\xec\xa0\xbc: ', '2.7.15+ (default, Nov 27 2018, 23:36:35) \n[GCC 7.3.0]')


보다시피 단순한 print에서도 차이가 난다.  
파이썬 2에서 print는 statement였다.  
파이썬 3에서 print()는 function이다.

print()처럼 쓰려고 해도 2개 이상 쓰자마자 tuple로 인식해 버리고 만다.

파이썬 3가 배포되면서 2에서 3으로 바로 넘어가기 어려워서 였는지 2와 3 모두 쓸 수 있는 방법들이 만들어 졌다.

~3.0.0이 배포된 게 2008년이고 이미 2.6이 있던 시기다. 그런데 벌써 10년이 넘게 매해 한 번씩은 파이썬 2를 작게라도 업데이트해준다. 슬슬 파이썬 2 지원을 끊어줬으면 한다.~

파이썬 공식 도큐먼트에 공개된 2+3 호환 코드 작성 단계는 아래와 같다.
1. 파이썬 2.7과 호환되면 된다.
2. 코드는 잘 테스트해보도록 하자.
3. 파이썬 2와 3의 차이를 미리 공부해 두자.
4. future 라이브러리를 활용하자.
5. pylint 툴로 코드를 점검해 보자.
6. caniusepython3 스크립트로 파이썬 3 사용을 막는 dependency를 찾아보자.
7. 앞으로도 파이썬 2와 3 모두에 쓸 수 있는 상태를 유지하자.
8. 변수 타입을 static하게 해줌으로써 해당 타입이 2와 3 모두에 쓰일 수 있게 하자.

## 함수 호환

### print

In [2]:
# 파이썬 2
print 'Hello'

# 파이썬 2+3
print('Hello')

# 파이썬 2
print 'Hello', 'world'

# 파이썬 3?
print('Hello', 'world')

Hello
Hello
Hello world
('Hello', 'world')


다시 print로 돌아오면 4번째 상황에서 문제가 일어남을 알 수 있다. 이것을 해결하기 위한 것이 future 라이브러리이다.

In [3]:
# 파이썬 2+3
from __future__ import print_function

print('Hello', 'world')

Hello world


무사히 print()가 불러와졌다!

다른 상황들을 살펴보자.

### raise

In [4]:
# 파이썬 2
raise ValueError, "에러가 일어나 버렸구나."

ValueError: ignored

In [5]:
# 파이썬 2+3
raise ValueError("에러가 일어나 버렸어.")

ValueError: ignored

### exception

In [6]:
# 파이썬 2
try:
    ...
except ValueError, e:
    ...

SyntaxError: ignored

In [7]:
# 파이썬 2+3
try:
    ...
except ValueError as e:
    ...

SyntaxError: ignored

### division

In [8]:
# 파이썬 2
print(2 / 3)

# 파이썬 2+3
print(2 // 3)

0
0


In [0]:
# 파이썬 3
3 / 2    # 파이썬 3 환경에서는 1.5로 true divison이 실행됨.

In [9]:
# 파이썬 2+3
from __future__ import division

3 / 2

1.5

이외에도 `long`이나 `metaclass`, `dict iteration`, `unicode` 등 다양한 곳에서 차이가 있다.

하지만 모든 것을 커버할 수 없으니 **필요하다면** 검색해보길 추천한다.

진짜로 거의 모든 영역에서 파이썬 2와 3은 충돌한다.

### 사용 라이브러리
- future: (2 &rarr; 3) 파이썬 3를 구현할 수 있다. 어째서인지 \_\_future__로 불러와야 한다.
- past: (3 &rarr; 2) 파이썬 2를 구현할 수 있다.
- builtins: 파이썬 빌트인 identifiers를 불러오므로 버젼과 상관없이 사용이 가능하다.

아래 라이브러리는 파이썬 3에는 있지만 파이썬 2에는 없다. 그래도 설치해서 사용이 가능하다고 한다.

- enum
- singledispatch: `functool` 안에 있었다.
- pathlib

## Import 호환

Exception를 활용하여 환경에 따라 맞춤 라이브러리를 불러올 수 있다. (`ImportError` 회피)

그러나 환경에 따라 기능이 같더라도 다른 라이브러리일 수 있기 때문에 `as`로 이름을 지정해주면 파이썬 2와 3에 상관없이 사용할 수 있다.

In [0]:
try:
    import urllib.request as urllib_request    # 파이썬 3
except:
    import urllib2 as urllib_request    # 파이썬 2

## 파이썬 2 내장함수 제거

파이썬 3는 파이썬 2의 내장함수(builtin function) 중 12개를 지원하지 않는다고 한다.  
실수로라도 해당 빌트인을 쓰지 않기 위해서 아래 코드를 쓰도록 한다.

In [11]:
from future.builtins.disabled import *

apply()

NameError: ignored

## 자동 호환

~솔직히 하나하나 다 따져가면서 코드를 짜고 싶지 않다.~

자동으로 파이썬 2를 3으로 바꿔주는 프로그램들이 있다.

### 2to3
[2.7: Automated Python 2 to 3 code translation](https://docs.python.org/2.7/library/2to3.html)

미리 저장해둔 fixer로 파이썬 2 코드를 3로 변환시켜준다.  
1:1 대응이기 때문에 유연성이 떨어질 수 있다.  
커맨드라인에 `2to3`하고 파일명을 적어주자.

파이썬이 공식으로 지원하는 프로그램이다.

[lib2to3](https://github.com/python/cpython/tree/master/Lib/lib2to3)라는 라이브러리로 API를 제공한다.

### Python-modernize
`lib2to3`를 랩핑하여 만들었다.

[Github](https://github.com/python-modernize/python-modernize)